In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("povarenok_recipes_2021_06_16.csv")
df = df.head(100)
df.head()

,url,name,ingredients
0,https://www.povarenok.ru/recipes/show/164365/,Густой молочно-клубничный коктейль,"{'Молоко': '250 мл', 'Клубника': '200 г', 'Сах..."
1,https://www.povarenok.ru/recipes/show/1306/,Рулетики,"{'Сыр твердый': None, 'Чеснок': None, 'Яйцо ку..."
2,https://www.povarenok.ru/recipes/show/10625/,"Салат ""Баклажанчик""","{'Баклажан': '3 шт', 'Лук репчатый': '2 шт', '..."
3,https://www.povarenok.ru/recipes/show/167337/,Куриные котлеты с картофельным пюре в духовке,"{'Фарш куриный': '800 г', 'Пюре картофельное':..."
4,https://www.povarenok.ru/recipes/show/91919/,Рецепт вишневой наливки,"{'Вишня': '1 кг', 'Водка': '1 л', 'Сахар': '30..."


In [3]:
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [15]:
"""
Здесь происходит экстрация отдельных продуктов питания из рецептов
Продукты консолидированы в один string для каждой строчки
"""
def extract_ingredients(ingredient_string):
    matches = re.findall(r"'([^']+)'(?=\s*:\s*)", ingredient_string)
    return matches

df['ingredients_extracted'] = df['ingredients'].apply(extract_ingredients)
df['ingredients_str'] = df['ingredients_extracted'].apply(lambda x: ' '.join(x))
print(f'{df.loc[0, 'name']} {df.loc[0, 'ingredients_str']}')

Густой молочно-клубничный коктейль Молоко Клубника Сахар


In [5]:
"""
Построение матрицы ингредиентов
На основе этой матрицы будет работать предложение
"""
count_vectorizer = CountVectorizer()
ingredient_matrix = count_vectorizer.fit_transform(df['ingredients_str'])

ingredient_vectors_df = pd.DataFrame(ingredient_matrix.toarray(), columns=count_vectorizer.get_feature_names_out())

result = pd.concat([df, ingredient_vectors_df], axis=1)
result

,url,name,ingredients,ingredients_extracted,ingredients_str,абрикос,авокадо,айва,ананас,апельсин,...,шампиньоны,шафран,шоколад,шпиг,щавель,яблоко,яблочный,ягода,яичный,яйцо
0,https://www.povarenok.ru/recipes/show/164365/,Густой молочно-клубничный коктейль,"{'Молоко': '250 мл', 'Клубника': '200 г', 'Сах...","[Молоко, Клубника, Сахар]",Молоко Клубника Сахар,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,https://www.povarenok.ru/recipes/show/1306/,Рулетики,"{'Сыр твердый': None, 'Чеснок': None, 'Яйцо ку...","[Сыр твердый, Чеснок, Яйцо куриное, Грейпфрут,...",Сыр твердый Чеснок Яйцо куриное Грейпфрут Лук ...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,https://www.povarenok.ru/recipes/show/10625/,"Салат ""Баклажанчик""","{'Баклажан': '3 шт', 'Лук репчатый': '2 шт', '...","[Баклажан, Лук репчатый, Помидор, Чеснок, Соль...",Баклажан Лук репчатый Помидор Чеснок Соль Майо...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,https://www.povarenok.ru/recipes/show/167337/,Куриные котлеты с картофельным пюре в духовке,"{'Фарш куриный': '800 г', 'Пюре картофельное':...","[Фарш куриный, Пюре картофельное, Желток яичны...",Фарш куриный Пюре картофельное Желток яичный Л...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,https://www.povarenok.ru/recipes/show/91919/,Рецепт вишневой наливки,"{'Вишня': '1 кг', 'Водка': '1 л', 'Сахар': '30...","[Вишня, Водка, Сахар, Кости, Корица, Гвоздика]",Вишня Водка Сахар Кости Корица Гвоздика,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,https://www.povarenok.ru/recipes/show/123901/,"Суп ""Грибное лукошко""","{'Масло растительное': '3 ст. л.', 'Грибы': '2...","[Масло растительное, Грибы, Шампиньоны, Картоф...",Масло растительное Грибы Шампиньоны Картофель ...,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
96,https://www.povarenok.ru/recipes/show/126929/,Пшенично-ржаной хлеб,"{'Мука пшеничная': '250 г', 'Мука ржаная': '25...","[Мука пшеничная, Мука ржаная, Дрожжи, Молоко, ...",Мука пшеничная Мука ржаная Дрожжи Молоко Вода ...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,https://www.povarenok.ru/recipes/show/143632/,Маринованные кабачки мгновенного приготовления,"{'Кабачок': '500 г', 'Соль': '1/2 ч. л.', 'Мас...","[Кабачок, Соль, Масло растительное, Мед, Чесно...",Кабачок Соль Масло растительное Мед Чеснок Укс...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,https://www.povarenok.ru/recipes/show/157466/,Жареные пирожки с мясом,"{'Вода': '625 мл', 'Мука пшеничная': '600 г', ...","[Вода, Мука пшеничная, Соль, Сахар, Масло раст...",Вода Мука пшеничная Соль Сахар Масло раститель...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
from sklearn.metrics.pairwise import cosine_similarity

def recommend_based_on_favorites(user_favorites, n_recommendations=15):
    indices = df.index[df['name'].isin(user_favorites)].tolist()
    
    if not indices:
        return pd.DataFrame()

    user_vectors = ingredient_matrix[indices]
    sim_scores = cosine_similarity(user_vectors, ingredient_matrix).mean(axis=0)

    similar_recipes = pd.Series(sim_scores, index=df.index).sort_values(ascending=False)
    recommendations = similar_recipes[~similar_recipes.index.isin(indices)].head(n_recommendations)
    return df.loc[recommendations.index, ['name', 'url']]

# Сюда можно вводить любимые блюда, чтобы получать рекоммендации основываясь на них
user_favorites = ['Густой молочно-клубничный коктейль', 'Салат "Баклажанчик"'] 
recommend_based_on_favorites(user_favorites)

,name,url
83,Любимые баклажаны по рецепту мамули,https://www.povarenok.ru/recipes/show/8227/
69,"Пирожки ""Малышки""",https://www.povarenok.ru/recipes/show/124175/
19,Горбуша с баклажанами по-китайски,https://www.povarenok.ru/recipes/show/94308/
79,Курица с рулетиками в духовке,https://www.povarenok.ru/recipes/show/152528/
65,Лазанья с тыквой и вялеными томатами,https://www.povarenok.ru/recipes/show/68424/
85,Голубцы с айвой по-Бакински,https://www.povarenok.ru/recipes/show/54169/
62,Суп из баранины с нутом,https://www.povarenok.ru/recipes/show/114130/
9,Молочное мороженое,https://www.povarenok.ru/recipes/show/96774/
10,Картошечка с грибами,https://www.povarenok.ru/recipes/show/23092/
32,"Овощная запеканка ""Всего понемногу""",https://www.povarenok.ru/recipes/show/145591/
